# allele_specific_analyses(10)

1/27/2021

run gatk (see one note for details)

now see if overlap with mpra results

In [1]:
import pandas as pd 
import os,sys, glob
import pybedtools
import scipy.stats as stats
import re
import numpy as np

In [2]:
def read_vcf(path):
    with open(path, 'r') as f:
        lines = f.readlines()
        lines = [l.strip().split('\t') for l in lines if not l.startswith('##')]
    df=  pd.DataFrame(lines[1: ],columns=lines[0]).rename(columns={'#CHROM': 'CHROM'})
    if not df['CHROM'][0].startswith('chr'):
        df['CHROM'] = 'chr' + df.CHROM.map(str)

    df['rowname'] =  df.CHROM + '_' + df.POS.map(str)
    print('read',path)
    print('shape',df.shape)
    return df

def preprocess_vcf(row,tissue):
    # get info quality metrics
    info_str = row['INFO']
    info_str_arr = re.split('[=;]',info_str)
    qual =  pd.Series(dict(zip(info_str_arr[::2],info_str_arr[1::2] )))
    qual = qual[['DP','SOR','FS']]
    # get 
    tissue_cols = [x for x in row.index if re.search('B[1-3]$',x) is not None]    #x.startswith(tissue[:min(len(tissue),4)])]
    if len(tissue_cols)==0:
        tissue_cols=['sample','sample']
#     print('tissue_cols',tissue_cols)
    try:
        sample_1_info = row[tissue_cols[0]].split(':')
        sample_2_info = row[tissue_cols[1]].split(':')

        qual['GT_1'] = sample_1_info[0]
        qual['GT_2'] = sample_2_info[0]
        qual['AD_1_ref'] = int(sample_1_info[1].split(',')[0])
        qual['AD_1_alt'] = int(sample_1_info[1].split(',')[1])
        qual['AD_2_ref'] = int(sample_2_info[1].split(',')[0])
        qual['AD_2_alt'] = int(sample_2_info[1].split(',')[1])
    except:
        print(row)
        print(tissue_cols)
        raise
    return qual
#     format_arr = row['FORMAT'].split(':')
#     
#     format_series = pd.Series()
#     for idx,sample in enumerate(tissue_cols):
#         format_arr_sample = [x+"_"+str(idx)for x in format_arr]
#         format_series=pd.concat([format_series,pd.Series(dict(zip(format_arr_sample,row[sample].split(':'))))])
#     return pd.concat([qual,format_series])

def filter_vcf(vcf_df):
    # filter depth count (DP)>=10
    vcf_df = vcf_df[vcf_df.DP.map(int)>=10]
    # the allele  only biallelic SNP sites (GT: 0/1) w
    vcf_df = vcf_df[(vcf_df.GT_1 =='0/1')& (vcf_df.GT_2 =='0/1')]
    
    # minimum reference or alternative allele count >=2 (AD)
    vcf_df['AD_1_min'] = vcf_df[['AD_1_ref','AD_1_alt']].min(axis=1)
    vcf_df = vcf_df[vcf_df.AD_1_min>=2]
    vcf_df['AD_2_min'] = vcf_df[['AD_2_ref','AD_2_alt']].min(axis=1)
    vcf_df = vcf_df[vcf_df.AD_2_min>=2]
    return vcf_df

In [3]:
save_dir = '../data/processed/vcf_files/analyses'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [4]:
# option1
lib_df1  = pd.read_csv('../data/external/GWAS/cancer_mpra_lib_snps_bedformat_121520.bed',sep='\t',header=None)
lib_df1.columns = ['chr','start','stop','rsid']
lib_df1['rowname'] = lib_df1.chr + '_' + lib_df1['start'].map(str)
lib_df_bed_df1 = lib_df1[['chr','start','stop','rowname']]
# option2
lib_df  = pd.read_csv('../data/external/GWAS/snp_list_combined.csv',index_col=0)
lib_df.columns = ['rsid','chr','start','disease']
lib_df['stop'] = lib_df['start'] + 1 
lib_df['rowname'] = lib_df.chr + '_' + lib_df['start'].map(str)
lib_df_bed_df = lib_df[['chr','start','stop','rowname']]
print('lib_df_bed_df',lib_df_bed_df.shape,'lib_df_bed_df1', lib_df_bed_df1.shape)
lib_df_bed_df = pd.concat([lib_df_bed_df,lib_df_bed_df1]).drop_duplicates()
print('lib_df_bed_df shape post merge',lib_df_bed_df.shape)
display(lib_df_bed_df[:5])
#common processing
lib_df_bed = pybedtools.BedTool.from_dataframe(lib_df_bed_df)
# print(lib_df.disease.unique())
# lib_df[:5]

lib_df_bed_df (35203, 4) lib_df_bed_df1 (5016, 4)
lib_df_bed_df shape post merge (38031, 4)


,chr,start,stop,rowname
0,chr1,44171211,44171212,chr1_44171211
1,chr1,8086527,8086528,chr1_8086527
2,chr1,108330356,108330357,chr1_108330356
3,chr5,44876507,44876508,chr5_44876507
4,chr5,50735307,50735308,chr5_50735307


In [34]:
mpra_info_df = pd.read_table('../data/external/GWAS/cancer_mpra_snps_results_120320.tsv',index_col=0)
mpra_info_df['rowname'] = 'chr'+mpra_info_df.Chr_37.map(str) + '_' +mpra_info_df.Start_37.map(str)
mpra_info_df[['Causal_SNP', 'rowname','disease']]

,Causal_SNP,rowname,disease
locus,,,
locus_1000,rs4866783,chr5_44876507,hmec
locus_1001,rs75282042,chr5_50735307,hmec
locus_1002,rs115010969,chr5_52535211,hmec
locus_1003,rs17343328,chr5_52615305,hmec
locus_1004,rs10940312,chr5_52667982,hmec
...,...,...,...
locus_996,rs16901937,chr5_44709141,hmec
locus_997,rs1866406,chr5_44809945,hmec
locus_999,rs10070037,chr5_44870237,hmec


In [10]:
mpra_res_df = pd.read_table('../data/external/GWAS/cancer_mpra_significant_hits_coords_1.27.21.tsv',index_col=0)
mpra_res_df['rowname'] = 'chr'+mpra_res_df.Chr_37.map(str) + '_' +mpra_res_df.Start_37.map(str)
# mpra_res_df = mpra_res_df.merge(lib_df[['Linked_SNP','rowname']],how='left',on='rowname' )
mpra_res_df[:5]

,Chr_37,Start_37,disease,hit_tissue,rowname
Causal_SNP,,,,,
rs111640187,5,82428995,airway,airway,chr5_82428995
rs34064842,6,27688625,"airway,hmec",airway,chr6_27688625
rs71546575,6,27805022,"airway,hmec",airway,chr6_27805022
rs35353359,6,28324378,"airway,hmec",airway,chr6_28324378
rs3095311,6,31051675,airway,airway,chr6_31051675


In [28]:
print(mpra_res_df.shape)
mpra_res_df.hit_tissue.unique()
# # from asATAC to tissue
mpra_tissue_mapper = {
    'Airway':'airway',
    'Astrocytes':'ast',
    'Colon':'colon',
    'Esophageal':'eso',
    'GM12878':'gm',
    'HMEC':'hmec',
    'GDSD6':'kc',
    'Melanocytes':'mc',
    'Ovarian':'ov',
    'Pancreas':'panc',
    'Prostate':'pros',
    'Renal':'renal',
    'Thyroid':'thy',
    'Uterine':'endo'
}
inv_mpra_tissue_mapper = {v: k for k, v in mpra_tissue_mapper.items()}
mpra_res_df['mapped_tissue'] = mpra_res_df.hit_tissue.map(inv_mpra_tissue_mapper )
mpra_res_df.mapped_tissue.value_counts()

(451, 6)


HMEC           147
Airway          68
Prostate        51
Colon           31
Ovarian         24
Melanocytes     23
Uterine         22
GDSD6           19
Pancreas        17
Thyroid         16
GM12878         16
Astrocytes      10
Esophageal       4
Renal            3
Name: mapped_tissue, dtype: int64

In [7]:
# mpra_rownames_sig = list(mpra_res_df.rowname.unique())
# len(mpra_rownames_sig)


In [71]:
# num_mpra_sig_hits = mpra_res_df.rowname.unique().shape[0]
num_lib_tested = lib_df.rowname.unique().shape[0]
# print(num_mpra_sig_hits, num_lib_tested)

# 1. atac

In [11]:
atac_dir = '../data/interim/merged/atac/'
atac_bed_files = glob.glob(os.path.join(atac_dir, '*bed'))
atac_bed_files_sel =sorted( [ '../data/interim/merged/atac/Renal_merged.bed',
 '../data/interim/merged/atac/Prostate_merged.bed',
 '../data/interim/merged/atac/Uterine_merged.bed',
 '../data/interim/merged/atac/GDSD6_merged.bed',
 '../data/interim/merged/atac/Pancreas_merged.bed',
 '../data/interim/merged/atac/Melanocytes_merged.bed',
 '../data/interim/merged/atac/Ovarian_merged.bed',
 '../data/interim/merged/atac/Bladder_merged.bed',
 '../data/interim/merged/atac/GDSD3_merged.bed',
 '../data/interim/merged/atac/Esophageal_merged.bed',
 '../data/interim/merged/atac/GM12878_merged.bed',
 '../data/interim/merged/atac/Astrocytes_merged.bed',
 '../data/interim/merged/atac/Colon_merged.bed',
 '../data/interim/merged/atac/GDSD0_merged.bed',
 '../data/interim/merged/atac/Thyroid_merged.bed',
 '../data/interim/merged/atac/Airway_merged.bed',
 '../data/interim/merged/atac/HMEC_merged.bed'])
atac_bed_files_sel

['../data/interim/merged/atac/Airway_merged.bed',
 '../data/interim/merged/atac/Astrocytes_merged.bed',
 '../data/interim/merged/atac/Bladder_merged.bed',
 '../data/interim/merged/atac/Colon_merged.bed',
 '../data/interim/merged/atac/Esophageal_merged.bed',
 '../data/interim/merged/atac/GDSD0_merged.bed',
 '../data/interim/merged/atac/GDSD3_merged.bed',
 '../data/interim/merged/atac/GDSD6_merged.bed',
 '../data/interim/merged/atac/GM12878_merged.bed',
 '../data/interim/merged/atac/HMEC_merged.bed',
 '../data/interim/merged/atac/Melanocytes_merged.bed',
 '../data/interim/merged/atac/Ovarian_merged.bed',
 '../data/interim/merged/atac/Pancreas_merged.bed',
 '../data/interim/merged/atac/Prostate_merged.bed',
 '../data/interim/merged/atac/Renal_merged.bed',
 '../data/interim/merged/atac/Thyroid_merged.bed',
 '../data/interim/merged/atac/Uterine_merged.bed']

In [30]:
# #testing
# atac_file = '/Users/mguo123/Documents/pan_omics_psych/data/interim/merged/atac/SL_D0_merged.bed'
# tissue_mapper = {
#     'H9_D2':'H9D2',
#     'SLC_D0':'SLC',
#     'SL_D0':'SL',
#     'Astrocytes':'AST1',
#     'H9_D10':'H9D10',
#     'H9_D0':'H9D0'
# }
# tissue = os.path.basename(atac_file).split('_merged')[0]
# atac_bed = pybedtools.BedTool(atac_file)
# lib_df_bed_atac = lib_df_bed.intersect(atac_bed).to_dataframe()
# lib_df_bed_atac  =lib_df_bed_atac[['name']].drop_duplicates().reset_index(drop=True)
# lib_df_bed_atac['tissue'] = tissue_mapper[tissue]
# lib_df_bed_atac.name[lib_df_bed_atac.name.str.startswith('chr3_503')]

In [31]:
# lib_atac_df.rowname[lib_atac_df.rowname.str.startswith('chr3_503')]

In [61]:
# aside get a table which lists for all the snps whether it's in an atac peak and which tissues

lib_atac_df = pd.DataFrame()
for atac_file in sorted(atac_bed_files_sel):
    tissue = os.path.basename(atac_file).split('_merged')[0]
    print(tissue)
#     if tissue not in tissue_mapper.keys():
#         print(tissue, 'not considered')
#         continue
    atac_bed = pybedtools.BedTool(atac_file)
    lib_df_bed_atac = lib_df_bed.intersect(atac_bed).to_dataframe()
    lib_df_bed_atac  =lib_df_bed_atac[['name']].drop_duplicates().reset_index(drop=True)
    lib_df_bed_atac['tissue'] = tissue#tissue_mapper[tissue]
    lib_atac_df = pd.concat([lib_atac_df, lib_df_bed_atac])
    print('lib_atac_df', lib_atac_df.shape)
    
lib_atac_df = lib_atac_df.groupby('name').agg({'tissue':'|'.join}).reset_index()
lib_atac_df.columns = ['rowname','atac_tissues']
lib_atac_df['bool_in_atac_pk'] = True
lib_atac_df.to_csv(os.path.join(save_dir,'lib_atac_annotation.csv'))

Airway
lib_atac_df (3067, 2)
Astrocytes
lib_atac_df (4414, 2)
Bladder
lib_atac_df (7692, 2)
Colon
lib_atac_df (11160, 2)
Esophageal
lib_atac_df (14751, 2)
GDSD0
lib_atac_df (17860, 2)
GDSD3
lib_atac_df (21137, 2)
GDSD6
lib_atac_df (24326, 2)
GM12878
lib_atac_df (27188, 2)
HMEC
lib_atac_df (30094, 2)
Melanocytes
lib_atac_df (31847, 2)
Ovarian
lib_atac_df (33669, 2)
Pancreas
lib_atac_df (37299, 2)
Prostate
lib_atac_df (39143, 2)
Renal
lib_atac_df (42699, 2)
Thyroid
lib_atac_df (46083, 2)
Uterine
lib_atac_df (50089, 2)


In [78]:
display(lib_atac_df[:5])
lib_atac_df.shape, lib_df_bed_df.shape

,rowname,atac_tissues,bool_in_atac_pk
0,chr10_101860630,Uterine,True
1,chr10_101860631,Uterine,True
2,chr10_101946033,Airway|Astrocytes|Bladder|Colon|Esophageal|GDS...,True
3,chr10_101989509,Airway|Astrocytes|Bladder|Colon|Esophageal|GDS...,True
4,chr10_101989510,Airway|Astrocytes|Bladder|Colon|Esophageal|GDS...,True


((7125, 3), (38031, 4))

In [63]:
vcf_files = sorted(glob.glob('../data/processed/vcf_files/atac/*vcf'))
vcf_files 

['../data/processed/vcf_files/atac/Airway_postvqsr.vcf',
 '../data/processed/vcf_files/atac/Astrocytes_postvqsr.vcf',
 '../data/processed/vcf_files/atac/Bladder_postvqsr.vcf',
 '../data/processed/vcf_files/atac/Colon_postvqsr.vcf',
 '../data/processed/vcf_files/atac/Esophageal_postvqsr.vcf',
 '../data/processed/vcf_files/atac/GDSD0_postvqsr.vcf',
 '../data/processed/vcf_files/atac/GDSD3_postvqsr.vcf',
 '../data/processed/vcf_files/atac/GDSD6_postvqsr.vcf',
 '../data/processed/vcf_files/atac/GM12878_postvqsr.vcf',
 '../data/processed/vcf_files/atac/HMEC_postvqsr.vcf',
 '../data/processed/vcf_files/atac/Melanocytes_postvqsr.vcf',
 '../data/processed/vcf_files/atac/Ovarian_postvqsr.vcf',
 '../data/processed/vcf_files/atac/Pancreas_postvqsr.vcf',
 '../data/processed/vcf_files/atac/Prostate_postvqsr.vcf',
 '../data/processed/vcf_files/atac/Renal_postvqsr.vcf',
 '../data/processed/vcf_files/atac/Thyroid_postvqsr.vcf',
 '../data/processed/vcf_files/atac/Uterine_postvqsr.vcf']

In [64]:
tissues = []
for vcf_file in vcf_files:
    tissue = os.path.basename(vcf_file).split('_post')[0]
    print(tissue)
    tissues.append(tissue)

Airway
Astrocytes
Bladder
Colon
Esophageal
GDSD0
GDSD3
GDSD6
GM12878
HMEC
Melanocytes
Ovarian
Pancreas
Prostate
Renal
Thyroid
Uterine


In [86]:
atac_tissue_mapper = dict(zip(tissues,tissues))
atac_tissue_mapper

{'Airway': 'Airway',
 'Astrocytes': 'Astrocytes',
 'Bladder': 'Bladder',
 'Colon': 'Colon',
 'Esophageal': 'Esophageal',
 'GDSD0': 'GDSD0',
 'GDSD3': 'GDSD3',
 'GDSD6': 'GDSD6',
 'GM12878': 'GM12878',
 'HMEC': 'HMEC',
 'Melanocytes': 'Melanocytes',
 'Ovarian': 'Ovarian',
 'Pancreas': 'Pancreas',
 'Prostate': 'Prostate',
 'Renal': 'Renal',
 'Thyroid': 'Thyroid',
 'Uterine': 'Uterine'}

In [66]:
### FROM TESTING
# vcf_df[:5]

hypergeometric test between two sets
- phyper=(overlap-1,list1,PopSize-list1,list2,lower.tail = FALSE, log.p = FALSE)
- scipy.stats.hypergeom.cdf(overlap, pop, list1, list2)

- 
phyper=(88,598,23000-598,5500,lower.tail = FALSE, log.p = FALSE)

In [39]:
def hypergeometric_test(x, M, n, N):
    """
    The hypergeometric distribution models drawing objects from a bin.
    - M is total number of objects
    - n is total number of Type I objects. 
    - x (random variate) represents the number of Type I objects in N drawn without replacement from the total population

    - http://en.wikipedia.org/wiki/Hypergeometric_distribution
    - https://www.biostars.org/p/66729/
    - http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.hypergeom.html
    - http://docs.scipy.org/doc/numpy/reference/generated/numpy.random.hypergeometric.html
    - http://stackoverflow.com/questions/6594840/what-are-equivalents-to-rs-phyper-function-in-python
    """

    assert n <= M
    assert x <= n
    assert N <= M
#     pv_le = stats.hypergeom.cdf(x+1, M, n, N)
    pv_gt = stats.hypergeom.sf(x-1, M, n, N)# 1-cdf sometimes more accurate
    return pv_gt#pv_gt pv_gt 


hyper geometric test 1
- check overlap between geneset of all asATAC snp locations with location with all of library snps over background of total # of snps? 
- check overlap between geneset of asATAC snps (tissue specific) in the library with the mpra hits in the library (tissue specific) 
- check overlap between geneset of asATAC snps (overall tissues) in the library with the mpra hits in the library all tissues)


In [90]:
%%time
asatac_lib_df = pd.DataFrame()
vcf_df_all = pd.DataFrame()
vcf_df_lib_all = pd.DataFrame()
vcf_df_lib_atac_all = pd.DataFrame()
for vcf_file in sorted(vcf_files):
    tissue = os.path.basename(vcf_file).split('_post')[0]
    print('************')
    print(tissue)
    vcf_df = read_vcf(vcf_file)
    vcf_df = pd.concat([vcf_df, vcf_df.apply(lambda x:preprocess_vcf(x,tissue),axis=1)], axis=1)
    num_vcf_hits = vcf_df.shape[0]
    print('num_vcf_hits PRE FILTER', num_vcf_hits)  
    vcf_df = filter_vcf(vcf_df)
    vcf_df['tissue'] = tissue
    num_vcf_hits_postfilt = vcf_df.shape[0]
    vcf_bed_df = vcf_df[['CHROM','POS','rowname']]
    vcf_bed_df['stop'] = vcf_df.POS.map(int) + 1
    vcf_bed_df = vcf_bed_df[['CHROM','POS','stop','rowname']]
    vcf_bed_df.columns = ['chr','start','stop','name']
    print('num_vcf_hits POST FILTER', num_vcf_hits_postfilt)      
    vcf_df_all = pd.concat([vcf_df_all, vcf_df])

    vcf_df_lib = vcf_df.merge(lib_df_bed_df, how='inner',on='rowname')
    vcf_df_lib_all = pd.concat([vcf_df_lib_all, vcf_df_lib])
    lib_hits = vcf_df_lib.rowname.unique()
    num_lib_hits = lib_hits.shape[0]
    print('background library hits that are asATAC', num_lib_tested,num_lib_hits, num_lib_hits/ num_lib_tested)
    
#     if tissue in mpra_tissue_mapper:
#         mpra_res_df_tissue = mpra_res_df[mpra_res_df.tissue==mpra_tissue_mapper[tissue]]
#         mpra_hits_tissue = mpra_res_df_tissue.rowname.unique()
#         num_mpra_hits_tissue= mpra_hits_tissue.shape[0]
# #         mpra_sig_df_lib = vcf_df.merge(mpra_res_df_tissue, how='inner',on='rowname')

#         num_mpra_asatac_hits = len(set(lib_hits).intersection(set(mpra_hits_tissue)))
#         asatac_lib_df_tissue=pd.DataFrame()
#         asatac_lib_df_tissue['rowname'] = sorted(set(lib_hits).intersection(set(mpra_hits_tissue)))
#         asatac_lib_df_tissue['tissue'] =  mpra_tissue_mapper[tissue]
#         asatac_lib_df = pd.concat([asatac_lib_df, asatac_lib_df_tissue])
#         # do a hypergeometric test between being asATAC and being an mpra hit over a background of being in the mpra dataset
    
#         print('mpra sig library hits in tissue ', num_mpra_hits_tissue)
#         print('overlap:', num_mpra_asatac_hits)
# #         oddsratio, pvalue = stats.fisher_exact([[num_lib_hits, num_lib_tested- num_lib_hits],
# #                                                 [num_mpra_hits, num_mpra_sig_hits - num_mpra_hits]])
# #         print('fisher for mpra and as annotation association', pvalue, oddsratio)
#         phyper =  hypergeometric_test(num_mpra_asatac_hits, num_lib_tested, num_mpra_hits_tissue, num_lib_hits)
#         #stats.hypergeom.cdf(num_mpra_asatac_hits, num_lib_tested, num_lib_hits, num_mpra_hits)
#         print('hypergeometric test between being asATAC and MPRA hit over backgroun of being in mpra_dataset', phyper )
#         phyper =  hypergeometric_test(num_mpra_asatac_hits, num_vcf_hits_postfilt, num_mpra_hits_tissue, num_lib_hits)
#         print('hypergeometric test between being asATAC and MPRA hit over backgroun of being in an asatac', phyper )
    
    ## filter through atac
    atac_tissue = atac_tissue_mapper[tissue]
    atac_bed_file = os.path.join(atac_dir, atac_tissue+'_merged.bed')
    if not os.path.exists(atac_bed_file):
        print('atac file',atac_bed_file, 'does not exist')
        continue
    atac_bed = pybedtools.BedTool(atac_bed_file)

    vcf_bed_df_atac = pybedtools.BedTool.from_dataframe(vcf_bed_df).intersect(atac_bed).to_dataframe()
    num_atac_vcf = vcf_bed_df_atac.name.unique().shape[0]
    print('atac filter', num_atac_vcf, 'out of', vcf_bed_df.shape[0], 'allele-specific atac', num_atac_vcf/vcf_bed_df.shape[0])
    lib_df_bed_atac = lib_df_bed.intersect(atac_bed).to_dataframe()
    lib_atac  = lib_df_bed_atac.name.unique()
    num_lib_atac = lib_atac.shape[0]
    print('atac filter', num_lib_atac, 'out of', num_lib_tested, 'mpra alleles tested', num_lib_atac/num_lib_tested)
#     mpra_atac = lib_df_bed_atac[lib_df_bed_atac.name.isin(mpra_rownames_sig)].name.unique()
#     num_mpra_atac = mpra_atac.shape[0]
#     print('atac filter', num_mpra_atac, 'out of', num_mpra_sig_hits, 'mpra alleles sig',num_mpra_atac/num_mpra_sig_hits)
    
    vcf_df_lib_atac = vcf_bed_df_atac.merge(lib_df_bed_df, how='inner',left_on='name',right_on='rowname')
    num_lib_hits_atac = vcf_df_lib_atac.rowname.unique().shape[0]
    vcf_df_lib_atac_all = pd.concat([vcf_df_lib_atac_all, vcf_df_lib_atac])
#     mpra_sig_df_lib_atac = vcf_bed_df_atac.merge(mpra_res_df, how='inner',left_on='name',right_on='rowname')
#     num_mpra_hits_atac = mpra_sig_df_lib_atac.rowname.unique().shape[0]
    
    print('background library hits-atac filt', num_lib_atac,num_lib_hits_atac, num_lib_hits_atac/ num_lib_atac)
#     print('mpra sig library hits-atac filt', num_mpra_atac,num_mpra_hits_atac, num_mpra_hits_atac/ num_mpra_atac)
#     if tissue in mpra_tissue_mapper:
#         mpra_hits_tissue_atac = set(mpra_hits_tissue).intersection(set(lib_atac))
#         num_mpra_hits_tissue_atac= mpra_hits_tissue.shape[0]
# #         mpra_sig_df_lib = vcf_df.merge(mpra_res_df_tissue, how='inner',on='rowname')

#         num_mpra_asatac_filtatac_hits = len(set(lib_hits).intersection(set(mpra_hits_tissue_atac)))
#         # do a hypergeometric test between being asATAC and being an mpra hit over a background of being in the mpra dataset

#         print('mpra sig library hits in tissue atac filt ', num_mpra_hits_tissue_atac)
#         print('overlap atac:', num_mpra_asatac_filtatac_hits)
# #         oddsratio, pvalue = stats.fisher_exact([[num_lib_hits, num_lib_tested- num_lib_hits],
# #                                                 [num_mpra_hits, num_mpra_sig_hits - num_mpra_hits]])
# #         print('fisher for mpra and as annotation association', pvalue, oddsratio)
#         phyper =  hypergeometric_test(num_mpra_asatac_filtatac_hits, num_atac_vcf, num_mpra_hits_tissue_atac, num_lib_atac)
#         #stats.hypergeom.cdf(num_mpra_asatac_hits, num_lib_tested, num_lib_hits, num_mpra_hits)
#         print('hypergeometric test between being asATAC and MPRA hit over backgroun of being asatac', phyper )


        

************
Airway
read ../data/processed/vcf_files/atac/Airway_postvqsr.vcf
shape (75646, 11)
num_vcf_hits PRE FILTER 75646


/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


num_vcf_hits POST FILTER 31211
background library hits that are asATAC 33066 853 0.025796891066352145
atac filter 21965 out of 31211 allele-specific atac 0.7037582903463523
atac filter 3067 out of 33066 mpra alleles tested 0.09275388616705982
background library hits-atac filt 3067 623 0.2031300945549397
************
Astrocytes
read ../data/processed/vcf_files/atac/Astrocytes_postvqsr.vcf
shape (59122, 11)


/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:1152: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


num_vcf_hits PRE FILTER 59122


...

num_vcf_hits POST FILTER 26163
background library hits that are asATAC 33066 658 0.01989959474989415
atac filter 18911 out of 26163 allele-specific atac 0.722814661927149
atac filter 3384 out of 33066 mpra alleles tested 0.10234077299945564
background library hits-atac filt 3384 482 0.14243498817966904
************
Uterine
read ../data/processed/vcf_files/atac/Uterine_postvqsr.vcf
shape (96367, 11)


/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:1152: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


num_vcf_hits PRE FILTER 96367


/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


num_vcf_hits POST FILTER 44282
background library hits that are asATAC 33066 1111 0.03359946773120426
atac filter 32958 out of 44282 allele-specific atac 0.7442753263176911
atac filter 4006 out of 33066 mpra alleles tested 0.12115163612169601
background library hits-atac filt 4006 840 0.20968547179231153
CPU times: user 1h 29min 1s, sys: 14.5 s, total: 1h 29min 15s
Wall time: 1h 28min 54s


we can see by the % that the mpra significant hits are overenriched compared to library background
- choice of library background can be changed to all gwas non cancer hits??? (will do next 

NOTE FISHER IS NOT THE RIGHT WAY TO DO IT!!! WIll need to fix ##TODO fix fisher #s


that means that MPRA signal is semi effected by 

In [91]:
vcf_df_all.to_csv(os.path.join(save_dir, 'vcf_df_all.csv'))
vcf_df_lib_all.to_csv(os.path.join(save_dir, 'vcf_df_lib_all.csv'))
vcf_df_lib_atac_all.to_csv(os.path.join(save_dir, 'vcf_df_lib_atac_all.csv'))


In [99]:
vcf_df_lib_atac_all.shape

(28936, 8)

In [111]:
asatac_lib_df = vcf_df_lib_atac_all.merge(vcf_df_all[['rowname','tissue']],how='left',on='rowname').drop_duplicates().sort_values('tissue').groupby('rowname').agg({'tissue':'|'.join}).reset_index()
asatac_lib_df.columns = ['rowname','tissue_asatac']
asatac_lib_df['bool_is_asatac'] = True
asatac_lib_df= asatac_lib_df.merge(lib_df_bed_df, how='left',on='rowname')
asatac_lib_df.to_csv(os.path.join(save_dir,'lib_asatac_annotation_postfilter.csv'))
print(asatac_lib_df.shape)
asatac_lib_df[:5]

(2595, 6)


,rowname,tissue_asatac,bool_is_asatac,chr,start,stop
0,chr10_102027407,GM12878|HMEC|Uterine,True,chr10,102027407,102027408
1,chr10_104228017,Colon|Pancreas,True,chr10,104228017,104228018
2,chr10_104239100,Bladder|Colon|Esophageal|GDSD6|Melanocytes|Pan...,True,chr10,104239100,104239101
3,chr10_104262628,Airway|Bladder|Colon|Esophageal|Melanocytes|Ov...,True,chr10,104262628,104262629
4,chr10_104263675,Airway|Bladder|Colon|Esophageal|Melanocytes|Ov...,True,chr10,104263675,104263676


# 1B. ATAC hypergeometric test

In [24]:
vcf_df_all = pd.read_csv(os.path.join(save_dir, 'vcf_df_all.csv'),index_col=0)
vcf_df_lib_all = pd.read_csv(os.path.join(save_dir, 'vcf_df_lib_all.csv'),index_col=0)
vcf_df_lib_atac_all = pd.read_csv(os.path.join(save_dir, 'vcf_df_lib_atac_all.csv'),index_col=0)
asatac_lib_df = pd.read_csv(os.path.join(save_dir,'lib_asatac_annotation_postfilter.csv'),index_col=0)


In [ ]:
#         phyper =  hypergeometric_test(num_mpra_asatac_hits, num_lib_tested, num_mpra_hits_tissue, num_lib_hits)
#         #stats.hypergeom.cdf(num_mpra_asatac_hits, num_lib_tested, num_lib_hits, num_mpra_hits)
#         print('hypergeometric test between being asATAC and MPRA hit over backgroun of being in mpra_dataset', phyper )
#         phyper =  hypergeometric_test(num_mpra_asatac_hits, num_vcf_hits_postfilt, num_mpra_hits_tissue, num_lib_hits)
#         print('hypergeometric test between being asATAC and MPRA hit over backgroun of being in an asatac', phyper )
 #         # do a hypergeometric test between being asATAC and being an mpra hit over a background of being in the mpra dataset
#     - M is total number of objects
#     - n is total number of Type I objects. 
#     - x (random variate) represents the number of Type I objects in N drawn without replacement from the total population


In [26]:
mpra_res_df
mpra_info_df[['Causal_SNP', 'rowname','disease']]
asatac_lib_df#.columns

,rowname,tissue_asatac,bool_is_asatac,chr,start,stop
0,chr10_102027407,GM12878|HMEC|Uterine,True,chr10,102027407,102027408
1,chr10_104228017,Colon|Pancreas,True,chr10,104228017,104228018
2,chr10_104239100,Bladder|Colon|Esophageal|GDSD6|Melanocytes|Pan...,True,chr10,104239100,104239101
3,chr10_104262628,Airway|Bladder|Colon|Esophageal|Melanocytes|Ov...,True,chr10,104262628,104262629
4,chr10_104263675,Airway|Bladder|Colon|Esophageal|Melanocytes|Ov...,True,chr10,104263675,104263676
...,...,...,...,...,...,...
2590,chr9_93940390,Bladder,True,chr9,93940390,93940391
2591,chr9_94877836,Airway|HMEC|Ovarian,True,chr9,94877836,94877837
2592,chr9_96213146,Uterine,True,chr9,96213146,96213147
2593,chr9_96213529,Uterine,True,chr9,96213529,96213530


In [54]:
# testing for given tissue
tissue_num_hits_dict = {}
hits
for tissue in sorted(mpra_tissue_mapper.keys()):
    print(tissue)
    mpra_tissue = mpra_tissue_mapper[tissue]
    lib_tested_rownames = set(mpra_info_df[mpra_info_df.disease.str.contains(mpra_tissue)].rowname.values)
    mpra_hit_rownames = set(mpra_res_df[mpra_res_df.mapped_tissue==tissue].rowname.values)
    asatac_hit_rowanmes = set(asatac_lib_df[asatac_lib_df.tissue_asatac.str.contains(tissue)].rowname.values)


    num_mpra_asatac_hits = len(mpra_hit_rownames.intersection(asatac_hit_rowanmes))
    num_lib_tested =lib_df1.shape[0] #len(lib_tested_rownames)
    num_mpra_hits = len(mpra_hit_rownames)
    num_asatac_hits = len(asatac_hit_rowanmes)
    print(num_mpra_asatac_hits,  num_lib_tested, num_mpra_hits, num_asatac_hits)
    phyper =  hypergeometric_test(num_mpra_asatac_hits, num_lib_tested, num_mpra_hits, num_asatac_hits)
    print('hypergeometric test between being asATAC and MPRA hit in a tissue over background of being in mpra_dataset', phyper )

    tissue_num_hits_dict[tissue]= num_mpra_asatac_hits
    

Airway
6 5016 68 747
hypergeometric test between being asATAC and MPRA hit in a tissue over background of being in mpra_dataset 0.9522384331479963
Astrocytes
2 5016 10 531
hypergeometric test between being asATAC and MPRA hit in a tissue over background of being in mpra_dataset 0.286650396582166
Colon
2 5016 31 608
hypergeometric test between being asATAC and MPRA hit in a tissue over background of being in mpra_dataset 0.9046046728017356
Esophageal
0 5016 4 600
hypergeometric test between being asATAC and MPRA hit in a tissue over background of being in mpra_dataset 1.0
GDSD6
0 5016 19 559
hypergeometric test between being asATAC and MPRA hit in a tissue over background of being in mpra_dataset 1.0
GM12878
0 5016 16 666
hypergeometric test between being asATAC and MPRA hit in a tissue over background of being in mpra_dataset 1.0
HMEC
5 5016 138 517
hypergeometric test between being asATAC and MPRA hit in a tissue over background of being in mpra_dataset 0.9991084649892193
Melanocytes


In [55]:
tissue_num_hits_dict

{'Airway': 6,
 'Astrocytes': 2,
 'Colon': 2,
 'Esophageal': 0,
 'GDSD6': 0,
 'GM12878': 0,
 'HMEC': 5,
 'Melanocytes': 2,
 'Ovarian': 0,
 'Pancreas': 1,
 'Prostate': 0,
 'Renal': 0,
 'Thyroid': 1,
 'Uterine': 3}

In [52]:
lib_df1.merge(mpra_info_df[['Causal_SNP', 'rowname','disease']],how='left',on='rowname')

,chr,start,stop,rsid,rowname,Causal_SNP,disease
0,chr1,46505785,46505785,rs71062735,chr1_46505785,rs71062735,hmec
1,chr1,46358009,46358010,rs4539075,chr1_46358009,NaN,NaN
2,chr1,46361176,46361177,rs7512395,chr1_46361176,NaN,NaN
3,chr1,88423171,88423172,rs7514001,chr1_88423171,NaN,NaN
4,chr1,108368897,108368902,rs11278684,chr1_108368897,NaN,NaN
...,...,...,...,...,...,...,...
5023,chr22,40436972,40436973,rs11704416,chr22_40436972,NaN,NaN
5024,chr22,38568832,38568833,rs738321,chr22_38568832,NaN,NaN
5025,chr22,30236242,30236243,rs2105870,chr22_30236242,NaN,NaN
5026,chr22,40948874,40948875,rs55662398,chr22_40948874,NaN,NaN


# 2. Hichip

In [112]:
hichip_peaks_dir = '../data/interim/merged/anchors_bed_sort'
hichip_bed_files = glob.glob(os.path.join(hichip_peaks_dir, '*_sort.bed'))
hichip_bed_files

['../data/interim/merged/anchors_bed_sort/COLO_SCR_DMSO_sort.bed',
 '../data/interim/merged/anchors_bed_sort/Ovarian_sort.bed',
 '../data/interim/merged/anchors_bed_sort/GDSD3_sort.bed',
 '../data/interim/merged/anchors_bed_sort/COLO_shMITF_DMSO_sort.bed',
 '../data/interim/merged/anchors_bed_sort/Melanocytes_sort.bed',
 '../data/interim/merged/anchors_bed_sort/Pancreas_sort.bed',
 '../data/interim/merged/anchors_bed_sort/Colon_sort.bed',
 '../data/interim/merged/anchors_bed_sort/SCC13-CTRLi_sort.bed',
 '../data/interim/merged/anchors_bed_sort/CAL27-CTRLi_sort.bed',
 '../data/interim/merged/anchors_bed_sort/Thyroid_sort.bed',
 '../data/interim/merged/anchors_bed_sort/Uterine_sort.bed',
 '../data/interim/merged/anchors_bed_sort/D6-p63i_sort.bed',
 '../data/interim/merged/anchors_bed_sort/GM12878_sort.bed',
 '../data/interim/merged/anchors_bed_sort/HMEC_sort.bed',
 '../data/interim/merged/anchors_bed_sort/A431-p63i_sort.bed',
 '../data/interim/merged/anchors_bed_sort/Astrocytes_sort.bed'

In [113]:
# for hichip_file in sorted(hichip_bed_files):
#     tissue = os.path.basename(hichip_file).split('_diffloop')[0]
#     print(tissue)
    

In [118]:
# aside get a table which lists for all the snps whether it's in a hichip anchor peak and which tissues
tissue_mapper = atac_tissue_mapper

lib_hichip_df = pd.DataFrame()
for hichip_file in sorted(hichip_bed_files):# sorted(hichip_bed_files):
    tissue = os.path.basename(hichip_file).split('_sort')[0]
    print(tissue)
    if tissue not in tissue_mapper.keys():
        print(tissue, 'not considered')
        continue
    hichip_bed = pybedtools.BedTool(hichip_file).to_dataframe()
    if type(hichip_bed['chrom'][0])==int:
        hichip_bed['chrom'] = 'chr' + hichip_bed.chrom.map(str)
        hichip_bed = pybedtools.BedTool.from_dataframe(hichip_bed)
    else:
        hichip_bed = pybedtools.BedTool(hichip_file)
    lib_df_bed_hichip = lib_df_bed.intersect(hichip_bed).to_dataframe()
    lib_df_bed_hichip  =lib_df_bed_hichip[['name']].drop_duplicates().reset_index(drop=True)
    lib_df_bed_hichip['tissue'] = tissue_mapper[tissue]
    lib_hichip_df = pd.concat([lib_hichip_df, lib_df_bed_hichip])
    print('lib_hichip_df', lib_hichip_df.shape)
    
lib_hichip_df = lib_hichip_df.groupby('name').agg({'tissue':'|'.join}).reset_index()
lib_hichip_df.columns = ['rowname','hichip_tissues']
lib_hichip_df['bool_in_hichip_pk'] = True
lib_hichip_df.to_csv(os.path.join(save_dir, 'lib_hichip_annotation.csv'))

A431-CTRLi
A431-CTRLi not considered
A431-p63i
A431-p63i not considered
Airway
lib_hichip_df (7094, 2)
Astrocytes
lib_hichip_df (19811, 2)
Bladder
lib_hichip_df (32719, 2)
CAL27-CTRLi
CAL27-CTRLi not considered
CAL27-p63i
CAL27-p63i not considered
COLO_SCR_DMSO
COLO_SCR_DMSO not considered
COLO_SCR_PLX
COLO_SCR_PLX not considered
COLO_shMITF_DMSO
COLO_shMITF_DMSO not considered
COLO_shMITF_PLX
COLO_shMITF_PLX not considered
Colon
lib_hichip_df (48275, 2)
D0-CTRLi
D0-CTRLi not considered
D0-p63i
D0-p63i not considered
D3-CTRLi
D3-CTRLi not considered
D3-p63i
D3-p63i not considered
D6-CTRLi
D6-CTRLi not considered
D6-p63i
D6-p63i not considered
Esophageal
lib_hichip_df (62259, 2)
GDSD0
lib_hichip_df (75640, 2)
GDSD3
lib_hichip_df (89399, 2)
GDSD6
lib_hichip_df (102456, 2)
GM12878
lib_hichip_df (115908, 2)
HMEC
lib_hichip_df (126392, 2)
Melanocytes
lib_hichip_df (135862, 2)
Ovarian
lib_hichip_df (150199, 2)
Pancreas
lib_hichip_df (162783, 2)
Prostate
lib_hichip_df (179240, 2)
Renal
lib_hi

In [119]:
vcf_files_hichip = glob.glob('../data/processed/vcf_files/hichip/*vcf')
vcf_files_hichip

['../data/processed/vcf_files/hichip/GDSD3_postvqsr.vcf',
 '../data/processed/vcf_files/hichip/GM12878_postvqsr.vcf',
 '../data/processed/vcf_files/hichip/Airway_postvqsr.vcf',
 '../data/processed/vcf_files/hichip/Bladder_postvqsr.vcf',
 '../data/processed/vcf_files/hichip/Astrocytes_postvqsr.vcf',
 '../data/processed/vcf_files/hichip/Esophageal_postvqsr.vcf',
 '../data/processed/vcf_files/hichip/Ovarian_postvqsr.vcf',
 '../data/processed/vcf_files/hichip/Melanocytes_postvqsr.vcf',
 '../data/processed/vcf_files/hichip/Colon_postvqsr.vcf',
 '../data/processed/vcf_files/hichip/Thyroid_postvqsr.vcf',
 '../data/processed/vcf_files/hichip/HMEC_postvqsr.vcf',
 '../data/processed/vcf_files/hichip/Uterine_postvqsr.vcf',
 '../data/processed/vcf_files/hichip/GDSD0_postvqsr.vcf',
 '../data/processed/vcf_files/hichip/GDSD6_postvqsr.vcf',
 '../data/processed/vcf_files/hichip/Renal_postvqsr.vcf',
 '../data/processed/vcf_files/hichip/Pancreas_postvqsr.vcf',
 '../data/processed/vcf_files/hichip/Prosta

In [120]:
for vcf_file in vcf_files_hichip:
    tissue = os.path.basename(vcf_file).split('_post')[0]
    print(tissue)

GDSD3
GM12878
Airway
Bladder
Astrocytes
Esophageal
Ovarian
Melanocytes
Colon
Thyroid
HMEC
Uterine
GDSD0
GDSD6
Renal
Pancreas
Prostate


In [121]:
hichip_tissue_mapper =atac_tissue_mapper

In [130]:
ashichip_lib_df = pd.DataFrame()
vcf_df_all_hichip = pd.DataFrame()
vcf_df_lib_all_hichip = pd.DataFrame()
vcf_df_lib_hichip_all_hichip = pd.DataFrame()
for vcf_file in sorted(vcf_files_hichip):
    tissue = os.path.basename(vcf_file).split('_post')[0]
    print('************')
    print(tissue)
    vcf_df = read_vcf(vcf_file)
    vcf_df = pd.concat([vcf_df, vcf_df.apply(lambda x:preprocess_vcf(x,tissue),axis=1)], axis=1)
    num_vcf_hits = vcf_df.shape[0]
    print('num_vcf_hits PRE FILTER', num_vcf_hits)  
    vcf_df = filter_vcf(vcf_df)
    vcf_df['tissue'] = tissue
    num_vcf_hits_postfilt = vcf_df.shape[0]
    vcf_bed_df = vcf_df[['CHROM','POS','rowname']]
    vcf_bed_df['stop'] = vcf_df.POS.map(int) + 1
    vcf_bed_df = vcf_bed_df[['CHROM','POS','stop','rowname']]
    vcf_bed_df.columns = ['chr','start','stop','name']
    print('num_vcf_hits POST FILTER', num_vcf_hits_postfilt)      
    vcf_df_all_hichip = pd.concat([vcf_df_all_hichip, vcf_df])

    vcf_df_lib = vcf_df.merge(lib_df_bed_df, how='inner',on='rowname')
    vcf_df_lib_all_hichip = pd.concat([vcf_df_lib_all_hichip, vcf_df_lib])
    lib_hits = vcf_df_lib.rowname.unique()
    num_lib_hits = lib_hits.shape[0]
    print('background library hits that are asHiChIP', num_lib_tested,num_lib_hits, num_lib_hits/ num_lib_tested)
    
#     if tissue in mpra_tissue_mapper:
#         mpra_res_df_tissue = mpra_res_df[mpra_res_df.tissue==mpra_tissue_mapper[tissue]]
#         mpra_hits_tissue = mpra_res_df_tissue.rowname.unique()
#         num_mpra_hits_tissue= mpra_hits_tissue.shape[0]
# #         mpra_sig_df_lib = vcf_df.merge(mpra_res_df_tissue, how='inner',on='rowname')

#         num_mpra_ashichip_hits = len(set(lib_hits).intersection(set(mpra_hits_tissue)))
#         ashichip_lib_df_tissue=pd.DataFrame()
#         ashichip_lib_df_tissue['rowname'] = sorted(set(lib_hits).intersection(set(mpra_hits_tissue)))
#         ashichip_lib_df_tissue['tissue'] =  mpra_tissue_mapper[tissue]
#         ashichip_lib_df = pd.concat([ashichip_lib_df, ashichip_lib_df_tissue])
#         # do a hypergeometric test between being asHiChIP and being an mpra hit over a background of being in the mpra dataset
    
#         print('mpra sig library hits in tissue ', num_mpra_hits_tissue)
#         print('overlap:', num_mpra_ashichip_hits)
# #         oddsratio, pvalue = stats.fisher_exact([[num_lib_hits, num_lib_tested- num_lib_hits],
# #                                                 [num_mpra_hits, num_mpra_sig_hits - num_mpra_hits]])
# #         print('fisher for mpra and as annotation association', pvalue, oddsratio)
#         phyper =  hypergeometric_test(num_mpra_ashichip_hits, num_lib_tested, num_mpra_hits_tissue, num_lib_hits)
#         #stats.hypergeom.cdf(num_mpra_ashichip_hits, num_lib_tested, num_lib_hits, num_mpra_hits)
#         print('hypergeometric test between being asHiChIP and MPRA hit over backgroun of being in mpra_dataset', phyper )
#         phyper =  hypergeometric_test(num_mpra_ashichip_hits, num_vcf_hits, num_mpra_hits_tissue, num_lib_hits)
#         print('hypergeometric test between being asHiChIP and MPRA hit over backgroun of being in an ashichip', phyper )
    
    ## filter through hichip
    if  tissue not in hichip_tissue_mapper:
        print(tissue,'not in hichip tissue mapper')
        continue
    hichip_tissue = hichip_tissue_mapper[tissue]
    hichip_bed_file = os.path.join(hichip_peaks_dir, hichip_tissue+'_sort.bed')
    if not os.path.exists(hichip_bed_file):
        print('hichip file',hichip_bed_file, 'does not exist')
        continue
    hichip_bed = pybedtools.BedTool(hichip_bed_file).to_dataframe()
    if type(hichip_bed['chrom'][0])==int:
        hichip_bed['chrom'] = 'chr' + hichip_bed.chrom.map(str)
        hichip_bed = pybedtools.BedTool.from_dataframe(hichip_bed)
    else:
        hichip_bed = pybedtools.BedTool(hichip_bed_file)
        
    vcf_bed_df_hichip = pybedtools.BedTool.from_dataframe(vcf_bed_df).intersect(hichip_bed).to_dataframe()
    num_hichip_vcf = vcf_bed_df_hichip.name.unique().shape[0]
    print('hichip filter', num_hichip_vcf, 'out of', vcf_bed_df.shape[0], 'allele-specific hichip', num_hichip_vcf/vcf_bed_df.shape[0])
    lib_df_bed_hichip = lib_df_bed.intersect(hichip_bed).to_dataframe()
    lib_hichip  = lib_df_bed_hichip.name.unique()
    num_lib_hichip = lib_hichip.shape[0]
    print('hichip filter', num_lib_hichip, 'out of', num_lib_tested, 'mpra alleles tested', num_lib_hichip/num_lib_tested)
#     mpra_hichip = lib_df_bed_hichip[lib_df_bed_hichip.name.isin(mpra_rownames_sig)].name.unique()
#     num_mpra_hichip = mpra_hichip.shape[0]
#     print('hichip filter', num_mpra_hichip, 'out of', num_mpra_sig_hits, 'mpra alleles sig',num_mpra_hichip/num_mpra_sig_hits)
    
    vcf_df_lib_hichip = vcf_bed_df_hichip.merge(lib_df_bed_df, how='inner',left_on='name',right_on='rowname')
    num_lib_hits_hichip = vcf_df_lib_hichip.rowname.unique().shape[0]
    vcf_df_lib_hichip_all_hichip = pd.concat([vcf_df_lib_hichip_all_hichip, vcf_df_lib_hichip])
#     mpra_sig_df_lib_hichip = vcf_bed_df_hichip.merge(mpra_res_df, how='inner',left_on='name',right_on='rowname')
#     num_mpra_hits_hichip = mpra_sig_df_lib_hichip.rowname.unique().shape[0]
    
    print('background library hits-hichip filt', num_lib_hichip,num_lib_hits_hichip, num_lib_hits_hichip/ num_lib_hichip)
#     print('mpra sig library hits-hichip filt', num_mpra_hichip,num_mpra_hits_hichip, num_mpra_hits_hichip/ num_mpra_hichip)
#     if tissue in mpra_tissue_mapper:
#         mpra_hits_tissue_hichip = set(mpra_hits_tissue).intersection(set(lib_hichip))
#         num_mpra_hits_tissue_hichip= mpra_hits_tissue.shape[0]
# #         mpra_sig_df_lib = vcf_df.merge(mpra_res_df_tissue, how='inner',on='rowname')

#         num_mpra_ashichip_filthichip_hits = len(set(lib_hits).intersection(set(mpra_hits_tissue_hichip)))
#         # do a hypergeometric test between being asHiChIP and being an mpra hit over a background of being in the mpra dataset

#         print('mpra sig library hits in tissue hichip filt ', num_mpra_hits_tissue_hichip)
#         print('overlap hichip:', num_mpra_ashichip_filthichip_hits)
# #         oddsratio, pvalue = stats.fisher_exact([[num_lib_hits, num_lib_tested- num_lib_hits],
# #                                                 [num_mpra_hits, num_mpra_sig_hits - num_mpra_hits]])
# #         print('fisher for mpra and as annotation association', pvalue, oddsratio)
#         phyper =  hypergeometric_test(num_mpra_ashichip_filthichip_hits, num_hichip_vcf, num_mpra_hits_tissue_hichip, num_lib_hichip)
#         #stats.hypergeom.cdf(num_mpra_ashichip_hits, num_lib_tested, num_lib_hits, num_mpra_hits)
#         print('hypergeometric test between being asHiChIP and MPRA hit over backgroun of being ashichip', phyper )



************
Airway
read ../data/processed/vcf_files/hichip/Airway_postvqsr.vcf
shape (1173693, 12)
CHROM                                                   chr8
POS                                                 25145740
ID                                                         .
REF                                                        A
ALT                                                        G
QUAL                                                  745.46
FILTER                                                  PASS
INFO       AC=2;AF=0.500;AN=4;AS_QD=18.17;BaseQRankSum=-8...
FORMAT                                        GT:AD:DP:GQ:PL
Air_B1                              0/1:8,12:20:99:415,0,184
Air_B3                              0/1:12,9:21:99:340,0,390
rowname                                        chr8_25145740
Name: 1007026, dtype: object
['Air_B1', 'Air_B3']


KeyboardInterrupt: 

In [ ]:
vcf_df_all_hichip.to_csv(os.path.join(save_dir, 'vcf_df_all_hichip.csv'))
vcf_df_lib_all_hichip.to_csv(os.path.join(save_dir, 'vcf_df_lib_all_hichip.csv'))
vcf_df_lib_hichip_all_hichip.to_csv(os.path.join(save_dir, 'vcf_df_lib_hichip_all_hichip.csv'))


In [ ]:
ashichip_lib_df = vcf_df_lib_hichip_all_hichip.merge(vcf_df_all[['rowname','tissue']],how='left',on='rowname').drop_duplicates()..sort_values('tissue').groupby('rowname').agg({'tissue':'|'.join}).reset_index()
ashichip_lib_df.columns = ['rowname','tissue_ashichip']
ashichip_lib_df['bool_is_ashichip'] = True
ashichip_lib_df= ashichip_lib_df.merge(lib_df_bed_df, how='left',on='rowname')
ashichip_lib_df.to_csv(os.path.join(save_dir,'lib_ashichip_annotation_post_filt.csv'))
